# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import DataScienceHelperLibrary as dsh

import numpy as np
import pandas as pd
import re

import nltk
nltk.download(['punkt', 'wordnet', 'stopwords'])
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

from sklearn.ensemble import AdaBoostClassifier

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import f1_score
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion

from sqlalchemy import create_engine

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ben\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ben\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ben\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load data from database
database = 'MyDisasterResponse.db'
table = 'MyMessage'

engine = create_engine('sqlite:///{}'.format(database))

df = pd.read_sql_table(table, engine)

In [3]:
dsh.AnalyzeDataFrame(df)

-------------------- Dataframe analysis started --------------------
Shape:  (22621, 41)
Number of duplicate rows:  0
-------------------- Analysis of Columns with NaN values --------------------
Columns having all values: 40, 97.56%
index, type: int64                     0.0
id, type: int64                        0.0
message, type: object                  0.0
genre, type: object                    0.0
related, type: int64                   0.0
request, type: int64                   0.0
offer, type: int64                     0.0
aid_related, type: int64               0.0
medical_help, type: int64              0.0
medical_products, type: int64          0.0
search_and_rescue, type: int64         0.0
security, type: int64                  0.0
military, type: int64                  0.0
child_alone, type: int64               0.0
water, type: int64                     0.0
food, type: int64                      0.0
shelter, type: int64                   0.0
clothing, type: int64              

In [9]:
df[df.original == np.nan]

,index,id,message,original,genre,related,request,offer,aid_related,medical_help,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report


Hmmm.... there were definitely missing values...

In [8]:
type(df.at[7408, 'original']) # example ID I radonmly found.

NoneType

In [12]:
df[df.original == None]

,index,id,message,original,genre,related,request,offer,aid_related,medical_help,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report


In [6]:
type(None)

NoneType

Hmmm.... I thought that None or np.nan would be interpretet as something like DBNULL. Instead it is a string value?! I didn't calculated that...

In [7]:
df[df.original == 'NoneType']

,index,id,message,original,genre,related,request,offer,aid_related,medical_help,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report


Double hmmm... I give up. I just intended to check the number of mising values before and after writing into database. But as you can see, currently I'm out of ideas ... ??

## Remark: Cleaning before modeling

As I mentioned at ETL 7.0, I didn't drop rows that were irrelevant because its possible that irrelevant rows have been added before selecting the data from database. 

So here at this step the data needs to be cleaned either way. My following steps:
- remove column 'original' as there are many missing values as explored during ETL and cannot be proven here as shown above :D
- remove rows when message starts with 'NOTES:'


In [7]:
df = dsh.RemoveColumnsByWildcard(df, 'original')

-------------------- Start finding and removing columns matchting to wildcards: ['original'] --------------------
Columns found to remove:  ['original']
-------------------- Finished removing columns matchting to wildcards --------------------


In [4]:
df[df['message'].str.startswith('NOTES:')]

,index,id,message,original,genre,related,request,offer,aid_related,medical_help,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
3062,3083,3539,NOTES: Personal. Not an emergency.,"prezidan preval mande,pou tout moun ki konn bw...",direct,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,1
3095,3116,3579,NOTES: aclet dry We are counting on you.,aclet dry. nous comptons sur vous.,direct,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
3099,3120,3583,NOTES: talking about gas prices,GEN SERI DE MESAJ FOK NOU TA BIEN PRECIZE YO L...,direct,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3162,3183,3651,NOTES: Historic earthquake in Haiti.,"istorik tranblemannte an ayiti svp,",direct,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3236,3257,3737,NOTES: They are requesting that somebody to pa...,Mesaj avek la otorite sil vou ple mezanmi pase...,direct,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3258,3279,3761,NOTES: Regular gossip or message sharing. Not ...,"Bonjour! jnfritz ,di odner mouin te re lel mou...",direct,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,1
3268,3289,3775,NOTES: personal request. Not an emergency.,Mwn vle yon ti infomasyon plis tj,direct,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3311,3332,3820,NOTES: regular gossip. Not an emergency.,Svp d yo ap bay yo pa rive sou nou nan jivna3 ...,direct,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3357,3378,3871,NOTES: Gossip. Not an emergency.,SE VANT NOU KI PLEN .PAVOYE MESAY ANKO SOU TE ...,direct,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3362,3383,3877,NOTES: Personal comment. Not an emergency.,i presizeman nan empas soya nou pa jwenn anyen...,direct,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
# Remove messages starting with 'NOTES:'

df = dsh.RemoveRowsWithValueInColumn(df, 'message', ['NOTES:'], option = 'startswith')

329 rows (ca. 1.45%) have been removed with value/s "['NOTES:']" in column "message"


In [21]:
ser = df.message

ser[ser.str.replace('[^a-zA-Z0-9]', ' ')]

message
Weather update   a cold front from Cuba that could pass over Haiti                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

In [7]:
df[df.message.str.contains('@')]

,index,id,message,original,genre,related,request,offer,aid_related,medical_help,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
304,307,373,The internet caf Net@le that's by the Dal road...,Cyber cafe net@le ki chita rout de dal tou pr ...,direct,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
900,909,1089,"We need food, water. .. . The kids are sick, f...","NOU NAN KOTE 28 RUE RICHARD Nou BEZWEN TANTE, ...",direct,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4179,4206,4788,"Help for Haiti food for gressier, medicine, an...","Help for haiti food for gressier, medicament a...",direct,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5546,5594,6333,I feel bien with my 2 years Jt@,je me sens bien avec 2an jta@,direct,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7241,7316,8192,0H 8@@ @2x @e8x3 xr,0Hlt 8@@ @2x @e8x3 xr a,direct,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7534,7609,8497,//// // @:@,//// // @lt : gt@,direct,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7583,7658,8553,Digicel @@ Digicel @@ Digicel I great you!,Digicel @@ @@ Digicel Digicel @@ @@ Digicel ...,direct,1,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
7646,7721,8621,I need a tente // / / // @ @ @ to sleep ha...,Mwen bezwen yon tant // / / // @ @ @ pou m...,direct,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8095,8170,9105,8 @9q@@ 5 m) 1A a!a ?aCYCLONE,8 @9q@@ 5 mamp ) 1A a!gtgta ?aCYCLONE,direct,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,1
8128,8206,9141,/''''''''@''''''''''''''''@ @''''' michou )) '...,/''''''''@''''''''''''''''@ @''''' michou ) ) ...,direct,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
df[df.index == 20814]

,index,id,message,original,genre,related,request,offer,aid_related,medical_help,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
20814,20985,24243,While the focus is to save lives and fight dis...,"Umer kot: Sindh: logon ko rehne ke liye ghar,s...",news,1,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0


In [31]:
print(list(df[df.index == 12224 ].message))

['PIDE                 .  ']


In [30]:
df[df.index == 12224]

,index,id,message,original,genre,related,request,offer,aid_related,medical_help,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
12224,12332,14439,PIDE .,PIDE . a,direct,1,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0


In [16]:
df.message.str.strip().str.len().sort_values()

12185        0
12222        0
12189        1
21688        6
22465        6
16504        6
12233        7
7534        11
7117        13
12131       15
6776        16
7241        19
12196       21
12149       22
12224       22
12280       23
9764        25
9800        27
12136       27
2947        28
8276        28
3779        28
9379        29
3442        29
7613        29
5714        29
6281        29
5630        29
2523        29
9852        29
         ...  
18051     2577
17964     2654
17150     2812
21548     2812
20627     2838
15813     2912
16054     2977
14710     3055
22571     3284
21970     3391
21480     3458
16108     3554
14707     3661
13648     3946
22178     3963
17248     4011
15803     4102
20638     4151
16806     4227
14680     4236
20731     4746
15884     4769
21044     5052
15095     5149
17299     5783
20619     6144
13841     6313
19363     6951
16007    10571
20814    10818
Name: message, Length: 22621, dtype: int64

In [4]:
df.genre.unique()

array(['direct', 'social', 'news'], dtype=object)

In [48]:
dfsub = df[(df.message.str.startswith('NOTE'))]['message']
#(df.related > 1) &
for val in list(dfsub):
    print(val)

NOTES: Personal. Not an emergency.
NOTES: aclet dry We are counting on you.
NOTES: talking about gas prices
NOTES: Historic earthquake in Haiti.
NOTES: They are requesting that somebody to pass by Butte Boyer #3 They don't have water food and Medicines
NOTES: Regular gossip or message sharing. Not an emergency.
NOTES: personal request. Not an emergency.
NOTES: regular gossip. Not an emergency.
NOTES: Gossip. Not an emergency.
NOTES: Personal comment. Not an emergency.
NOTES: Personal opinion. Not an emergency.
NOTES: Inquiry. No action needed at this time
NOTES: looking for work - construction
NOTES: Concerns about remainder of school year
NOTES: Gossip. Personal. Not an emergency.
NOTES: Discard. Not an emergency.
NOTES: Personal concerns and offer of help to ditribute goods.
NOTES: An opinion and request to the Gvt
NOTES: Comment and political opinion. No emergency.
NOTES: Statement. No emergency.
NOTES: The stench of death is overwhelming the citizens
NOTES: I already translated thi

In [9]:
'''
for i in range(dfsub.shape[0]):
    dsh.PrintLine('Row: ' + str(i))
    for j in range(dfsub.shape[1]):
        if dfsub.columns[j] in ['id', 'index']:
            continue
        if dfsub.iloc[i, j] == 0:
            continue
        print(dfsub.columns[j], ' - ', dfsub.iloc[i, j])
'''

"\nfor i in range(dfsub.shape[0]):\n    dsh.PrintLine('Row: ' + str(i))\n    for j in range(dfsub.shape[1]):\n        if dfsub.columns[j] in ['id', 'index']:\n            continue\n        if dfsub.iloc[i, j] == 0:\n            continue\n        print(dfsub.columns[j], ' - ', dfsub.iloc[i, j])\n"

In [10]:
X = dsh.SelectColumnsByWildcard(df, ['message'])

Y = dsh.RemoveColumnsByWildcard(df, ['index', 'id', 'message', 'original', 'genre'])

-------------------- Start finding and keeping columns matchting to wildcards: ['message'] --------------------
-------------------- Finished keeping columns matchting to wildcards: 1 --------------------
-------------------- Start finding and removing columns matchting to wildcards: ['index', 'id', 'message', 'original', 'genre'] --------------------
Columns found to remove:  ['index', 'id', 'message', 'genre']
-------------------- Finished removing columns matchting to wildcards --------------------


### 2. Write a tokenization function to process your text data

In [49]:
# 
regUrl = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
regChars = r'[^a-zA-Z0-9]'

supportedLanguages = [
    'english', 
#    'frensh'
]

def Tokenize(text):
    
    text = text.lower()
    
    #text = re.sub(regUrl, " ", text)
    text = re.sub(regChars, " ", text)
    
    tokens = word_tokenize(text)
    
    
    
    lemmatizer = WordNetLemmatizer()
    
    cleanTokens = []
    for tok in tokens:
        cleanTok = lemmatizer.lemmatize(tok).strip()
        #cleanTok = [PorterStemmer().stem(tok) for tok in cleanTok]
        
        try:
            for lang in supportedLanguages:
                if cleanTok in stopwords.words(lang):
                    continue
        except:
            print('Error during stop word handling with language: ', lang)
        
        if len(cleanTok) > 0:
            cleanTokens.append(cleanTok)
    
    return cleanTokens
        
        

In [50]:
Tokenize('0H4Z(Z x(@zf0 ?!?@x PrxP0c6 ')

['0h4z', 'z', 'x', 'zf0', 'x', 'prxp0c6']

In [41]:
vals = list(df[df.genre == 'social'].message[:10])
for val in vals:
    dsh.PrintLine()
    print(val)
    print(Tokenize(val))
    dsh.PrintLine()

--------------------  --------------------
My thoughts and prayers go out to all the lives that were affected today by the earthquake in Haiti. I will keep you all in my prayers!
['thought', 'prayer', 'go', 'life', 'affected', 'today', 'earthquake', 'haiti', 'keep', 'prayer']
--------------------  --------------------
--------------------  --------------------
I m sorry for the poor people in Haiti tonight. That being said I d love to experience an earthquake just once. In a big field of course.
['sorry', 'poor', 'people', 'haiti', 'tonight', 'said', 'love', 'experience', 'earthquake', 'big', 'field', 'course']
--------------------  --------------------
--------------------  --------------------
RT selenagomez UNICEF has just announced an emergency alert for the people of Haiti who were hit by a 7.0 earthquake and a tsunami.. ..
['rt', 'selenagomez', 'unicef', 'ha', 'announced', 'emergency', 'alert', 'people', 'haiti', 'hit', '7', '0', 'earthquake', 'tsunami']
--------------------  ---

### 3. Build a machine learning pipeline
- You'll find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [64]:
# pipeline = Pipeline([
#         ('features', FeatureUnion([

#             ('text_pipeline', Pipeline([
#                 ('vect', CountVectorizer(tokenizer = Tokenize)),
#                 ('tfidf', TfidfTransformer())
#             ])),

#             #('starting_verb', StartingVerbExtractor())
#         ])),

#         ('clf', MultiOutputClassifier())
#     ])

classifier = AdaBoostClassifier()
pipeline = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultiOutputClassifier(classifier)),
                    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [65]:
XTrain, XTest, yTrain, yTest = dsh.SplitDataTrainTest(X, Y)

In [66]:
print(XTrain.shape)
print(XTest.shape)
print(yTrain.shape)
print(yTest.shape)

(18351, 1)
(7865, 1)
(18351, 36)
(7865, 36)


In [69]:
pipeline.fit(X = XTrain, y = yTrain)

ValueError: Found input variables with inconsistent numbers of samples: [1, 18351]

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = {
        'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
        'features__text_pipeline__vect__max_df': (0.5, 0.75, 1.0),
        'features__text_pipeline__vect__max_features': (None, 5000, 10000),
        'features__text_pipeline__tfidf__use_idf': (True, False),
        'clf__n_estimators': [50, 100, 200],
        'clf__min_samples_split': [2, 3, 4],
        'features__transformer_weights': (
            {'text_pipeline': 1, 'starting_verb': 0.5},
            {'text_pipeline': 0.5, 'starting_verb': 1},
            {'text_pipeline': 0.8, 'starting_verb': 1},
        )
    }

cv = GridSearchCV(pipeline, param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.